# HW3: 新規ユーザーの初日の行動と翌日起動の関係を調べる

user_data.user_masterとstat.user_article_statを使い、2016年1月1日入会ユーザーについて、各ユーザーの入会後24時間以内行動データ（以下のような表）と次の24時間から48時間でアプリを起動したかどうかをを作成してください。スライドP.91の質問に答えてください。

## 質問
ダウンロード翌日（ダウンロードしてから24時間後~48時間後)起動する人とそうでない人について、初日にとった以下の行動の回数で大きな差異が見られますか？

もし差異があるとしたら、どの行動ですか？

* app_open（アプリの起動）
* topic_follow　（テーマをフォローした）
* article_show　（記事タイトルを見た）
* article_read　（記事本文を閲覧した）


In [3]:
%matplotlib inline
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

import psycopg2

from __future__ import print_function

<H2>データベースへのアクセスを確立する</H2>
* dbnameはアクセスするデータベースの名前を指定
* userはユーザー名
* passwordはパスワード
* hostはデータベースがあるサーバーのアドレス
* portは通信する際のポート番号（今回はRedshiftなので5439。実務で使う場合はデータベースエンジニアなど管理者に確認）

In [4]:
db_params = {
    'dbname': 'kameliodemo',
    'user': 'dsbootcamp',
    'password': 'July2017',
    'host': "sql-training.crijpbsngns8.ap-northeast-1.redshift.amazonaws.com",
    'port': 5439
}

con = psycopg2.connect(**db_params)

# ここからコードを書いてください。

In [15]:
your_sql = """
WITH exec_nextday as (
        SELECT u.user_id,
               CASE WHEN count(0) > 0 THEN '1' 
                    ELSE '0'
                    END as tomorrow_flg
          FROM stat.user_article_stat as s
               INNER JOIN user_data.user_master as u
               ON u.user_id = s.user_id
         WHERE (s.created_at >= u.created_at + interval '24 hours')
           AND (s.created_at < u.created_at + interval '48 hours')
         GROUP BY u.user_id
        HAVING count(s.id) > 0
    ),
    exec_today as (
        SELECT s.user_id,
               SUM(CASE WHEN s.behave_type = 'app_open' THEN 1 ELSE 0 END) as app_open,
               SUM(CASE WHEN s.behave_type like 'topic_follow%' THEN 1 ELSE 0 END) as topic_follow,
               SUM(CASE WHEN s.behave_type = 'article_show' THEN 1 ELSE 0 END) as article_show,
               SUM(CASE WHEN s.behave_type = 'article_read' THEN 1 ELSE 0 END) as article_read
          FROM stat.user_article_stat as s
               INNER JOIN user_data.user_master as u
               ON u.user_id = s.user_id
         WHERE (u.created_at <= s.created_at)
           AND (s.created_at <= u.created_at + interval '24 hours')
         GROUP BY s.user_id
    ),
    user_activity as (
        SELECT t.user_id,
               a.gender,
               a.age,
               t.app_open,
               t.topic_follow,
               t.article_show,
               t.article_read,
               CASE WHEN n.tomorrow_flg = '1' THEN '1' ELSE '0' END as tomorrow_flg
          FROM exec_today as t
               LEFT OUTER JOIN exec_nextday as n
               ON t.user_id = n.user_id
               LEFT OUTER JOIN user_data.user_attribute as a
               ON t.user_id = a.user_id
    )

SELECT gender,
       tomorrow_flg,
       SUM(app_open) as app_open,
       SUM(topic_follow) as topic_follow,
       SUM(article_show) as article_show,
       SUM(article_read) as article_read
  FROM user_activity 
 WHERE gender is not null
 GROUP BY gender, tomorrow_flg
 ORDER BY gender, tomorrow_flg
"""

In [16]:
user_sql = psql.read_sql(con=con, sql=your_sql)
user_sql.head(n=100)

,gender,tomorrow_flg,app_open,topic_follow,article_show,article_read
0,female,0,92,24,2467,183
1,female,1,256,8,7391,264
2,male,0,362,82,9817,426
3,male,1,1474,160,30397,1433


女性の場合（翌日もアプリを使用していた人の初日の行動割合）

・app_open（アプリの起動）256件／348件（73.5%）

・article_show（記事タイトルを見た）7391件／9858件（74.9%）

・article_read（記事本文を閲覧した）264件／447件（59%）

・topic_follow（テーマをフォローした）8件／32件（25%）



男性の場合（翌日もアプリを使用していた人の初日の行動割合）

・app_open（アプリの起動）1474件／1936件（76.1%）

・article_show（記事タイトルを見た）30397件／40214件（75.5%）

・article_read（記事本文を閲覧した）1433件／1859件（77%）

・topic_follow（テーマをフォローした）160件／242件（66.1%）



考察：

男女別で比較すると、女性の場合には下記の傾向が識別された。
女性ユーザーは母集団数自体が少ないが、女性を対象とした「記事の内容」について、改善すべき可能性がある。

・初日に「記事本文を閲覧した」ユーザーの内、翌日もアプリを利用したユーザーは59%

・初日に「テーマをフォローした」ユーザーの内、翌日もアプリを利用したユーザーは25%

また、男性・女性ともに、初日に「記事タイトルを見た」件数に対して「記事本文を見た」件数が5%以下である。「記事タイトル」が、ユーザーを「記事本文」へアクセスさせる動機づけに不十分であり、「記事タイトル」の見せ方を改善すべき可能性がある。



